# Generate prompt logs, with consolidated code and embedded ChatGPT instructions

### ChatGPT separates code blocks from instructions. What if we could put all the instructions in with all the code that's been generated. In addition, let's get a log of all our prompts because ChatGPT probably missed something (or maybe we did).

### Here, instead of providing asks and responses, I'll just provide a log with some spurious items removed. The asks may not be of interest to everyone but the flow may be.

In [3]:
# 1. create a solution  with GCP that sends out text messages to a list of phone numbers on a regular basis with a message of "Time to submit your times sheets"
# 2. enhance solution to get information from a bigquery database table with fields of name and number. solution should not use twilio
# 3. enhance solution to use gcp secrets manager
# 4. provide a gcloud based solution to deploy the google cloud scheduler job to trigger the cloud function
# 5. do i really need a pub sub topic to have cloud scheduler trigger a cloud function?
# 6. please provide a python based solution for using google cloud scheduler to trigger the cloud function. include the commented out instructions in the code block
# 7. consolidate all code blocks to date and include commented out instructions for using the code
# 8. continue
# 9. provide a log of all my prompts. number each prompt. put results in a code block.


In [ ]:
# Author: XXXXX
# Creation Date: 2023-04-04 (Pacific Time)

# Enhance solution to get information from a BigQuery table and send SMS without using Twilio
import os
import plivo
from google.cloud import bigquery, secretmanager
from flask import abort

def get_secret(secret_name):
    client = secretmanager.SecretManagerServiceClient()
    project_id = os.environ.get("GCP_PROJECT")
    secret_version_path = client.secret_version_path(project_id, secret_name, "latest")
    response = client.access_secret_version(request={"name": secret_version_path})
    return response.payload.data.decode("UTF-8")

def send_timesheet_reminder(request):
    # Verify the request is a POST request
    if request.method != 'POST':
        return abort(405)

    # Instructions for setting up the Cloud Function and related services:
    # 1. Create a Plivo account and get your Auth ID, Auth Token, and a Plivo phone number.
    # 2. Create a new Google Cloud project or use an existing one.
    # 3. Enable Cloud Functions, Cloud Scheduler, BigQuery, and Secrets Manager APIs in your GCP project.
    # 4. Create a BigQuery dataset and table to store the phone numbers and names.
    # 5. Create secrets in GCP Secrets Manager for the Plivo Auth ID, Auth Token, and Plivo phone number.
    # 6. Create and deploy the Cloud Function using the consolidated code block provided above.
    # 7. Grant the Cloud Function access to the secrets.
    # 8. Create a Cloud Scheduler job to directly trigger the Cloud Function using an HTTP request.
    #    (See the Python-based solution for creating a Cloud Scheduler job below)

    message = "Time to submit your timesheets"

    # Get Plivo credentials from Secrets Manager
    auth_id = get_secret("plivo_auth_id")
    auth_token = get_secret("plivo_auth_token")
    plivo_phone_number = get_secret("plivo_phone_number")

    client = plivo.RestClient(auth_id, auth_token)

    # Fetch phone numbers from BigQuery
    bigquery_client = bigquery.Client()
    query = """
        SELECT name, number
        FROM `<your_project_id>.timesheet_reminder.employees`
    """
    query_job = bigquery_client.query(query)
    results = query_job.result()

    for row in results:
        name, number = row["name"], row["number"]
        personalized_message = f"Hi {name}, {message}"
        client.messages.create(
            src=plivo_phone_number,
            dst=number,
            text=personalized_message
        )

# Python-based solution for creating a Cloud Scheduler job to trigger the Cloud Function

# Instructions:
# 1. Install the google-cloud-scheduler package using pip:
#    pip install google-cloud-scheduler
# 2. Set the environment variable for your Google Cloud project ID:
#    export GOOGLE_CLOUD_PROJECT=<your_project_id>
# 3. Replace <your_trigger_url> with the Trigger URL from your Cloud Function.
# 4. Update the schedule as needed using Unix-cron format.
# 5. Set the desired time zone. If not specified, the default time zone is "UTC".

import os
from google.cloud import scheduler_v1

def create_cloud_scheduler_job():
    project_id = os.environ.get("GOOGLE_CLOUD_PROJECT")
    location = "us-central1"  # Update the location if needed
    job_name = "projects/{}/locations/{}/jobs/timesheetReminderJob".format(project_id, location)
    trigger_url = "<your_trigger_url>"
    schedule = "0 9 * * 5"  # Example: Every Friday at 9 AM
    time_zone = "America/Los_Angeles"

    client = scheduler_v1.CloudSchedulerClient()
    parent = f"projects/{project_id}/locations/{location}"

    # Create the Cloud Scheduler job
    job = {
        "name": job_name,
        "http_target": {
            "uri": trigger_url,
            "http_method": scheduler_v1.HttpMethod.POST,
        },
        "schedule": schedule,
        "time_zone": time_zone,
    }

    response = client.create_job(request={"parent": parent, "job": job})
    print(f"Created job: {response.name}")

# Execute the function to create the Cloud Scheduler job
create_cloud_scheduler_job()

